In [1]:
"""
Combine the non-prior background all-sky scan TS
with the GBM/fake healpix prior
to get the maximum TSs

This is a jupyter test.
The actual work is done with `02_get_background_TS_with_healpix.py`
"""

'\nCombine the non-prior background all-sky scan TS\nwith the GBM/fake healpix prior\nto get the maximum TSs\n\nThis is a jupyter test.\nThe actual work is done with `02_get_background_TS_with_healpix.py`\n'

In [2]:
import os
import sys
print("Python version: ", end=' ')
print(sys.version)

import numpy as np
import healpy as hp
# import histlite as hl
import csky as cy
import pandas as pd
from scipy import sparse

# import matplotlib.pyplot as plt
# from matplotlib import cm
# import matplotlib.colors as colors
############# comment out below two lines on clusters ##########
# %matplotlib inline
# %matplotlib notebook
################################################################
from glob import glob
timer = cy.timing.Timer()
time = timer.time

###### Local Import ######
sys.path.append('../../')
from greco_grb.scripts import SETTING
paths = SETTING.PATH()
print(paths)
LOCATION = paths.LOCATION
USER = paths.USER
ICDATA_DIR = paths.ICDATA_DIR
DATA_DIR = paths.DATA_DIR
ANA_DIR = paths.ANA_DIR

from greco_grb.scripts.utils import *

Python version:  3.7.3 (default, Mar 27 2019, 22:11:17) 
[GCC 7.3.0]

#### Working at GT ####

LOCATION: 	 GT
USER: 	 cchen641
ICDATA_DIR: 	 /storage/home/hhive1/cchen641/data/icecube/data/analyses/online_GRECO/candidate-002-p10
DATA_DIR: 	 /storage/home/hhive1/cchen641/data/icecube/data/greco_grb/data
ANA_DIR: 	 /storage/home/hhive1/cchen641/data/icecube/data/greco_grb/data/csky_output



In [3]:
import argparse

######################### CONFIGURE ARGUEMENTS #############################
# p = argparse.ArgumentParser(description="Background Trials",
#                             formatter_class=argparse.RawTextHelpFormatter)
# p.add_argument("--grb_name_idx_start", default=0, type=int, help="Starting index of one GRB")
# p.add_argument("--grb_name_idx_end", default=1, type=int, help="Ending index of one GRB")
# p.add_argument("--tw_in_second", default=10, type=int, help="Length of the time window in seconds")
# args = p.parse_args()
###########################################################################


### testing on jupyter ###

class args:
    grb_name_idx_start = 0    # [start, end)   [inclusive, exclusive)
    grb_name_idx_end = 2
    # grb_name = "GRB180423A"    # real healpix example
    # grb_name = "GRB190611B"    # fake healpix example
    tw_in_second = 100

##########################

In [4]:
print("\n===== Loading no-healpix df =====\n")
df = pd.read_pickle(DATA_DIR+"/grbwebgbm/grbweb_gbm_noHealpix_2268.pkl")
# realHealpix: df.gbm & df.hasHealpix
# fakeHealpix: ~(df.gbm & df.hasHealpix)
# df.head()


===== Loading no-healpix df =====



In [5]:
files = glob(ANA_DIR + "/allsky_scan/no_prior_versatile/tw{}/GRB180423A_batchSize2500_batchIndex*_tw{}.npz".format(args.tw_in_second, args.tw_in_second))
files.sort(key=lambda x: int(x[x.find("batchIndex") + len("batchIndex"): x.find("_tw")]))


In [6]:
################ for testing  #############
files = files[:4]
###########################################

In [7]:
files

['/storage/home/hhive1/cchen641/data/icecube/data/greco_grb/data/csky_output/allsky_scan/no_prior_versatile/tw100/GRB180423A_batchSize2500_batchIndex0_tw100.npz',
 '/storage/home/hhive1/cchen641/data/icecube/data/greco_grb/data/csky_output/allsky_scan/no_prior_versatile/tw100/GRB180423A_batchSize2500_batchIndex1_tw100.npz',
 '/storage/home/hhive1/cchen641/data/icecube/data/greco_grb/data/csky_output/allsky_scan/no_prior_versatile/tw100/GRB180423A_batchSize2500_batchIndex2_tw100.npz',
 '/storage/home/hhive1/cchen641/data/icecube/data/greco_grb/data/csky_output/allsky_scan/no_prior_versatile/tw100/GRB180423A_batchSize2500_batchIndex3_tw100.npz']

In [8]:
def getCombinedMaxTS(healpix, scan):
    """
    Combine the non-prior background TS scan with the healpix prior
    then return the maximum TS
    
    Parameters
    ----------
    healpix : array_like
        A prior healpix map. shape [hp.nside2pix(64), ]
        
    scan : array_like (scipy.sparse matrix)
        One non-prior background scan (shape [1, hp.nside2pix(64)])
    
    Returns
    -------
    TS : scalar
        The maximum TS from the combined background.
    
    See also
    -------
    N/A
    """
    healpix = np.maximum(healpix, 1e-15)
    ts = scan.copy()
    # non-overlapping region will have negative TS anyways
    ts.data += 2. * (np.log(healpix[scan.indices]) - np.log(np.max(healpix)))
    ts.data[~np.isfinite(ts.data)] = 0
    return np.max(ts)

In [15]:
df.index

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            2287, 2288, 2289, 2290, 2291, 2292, 2293, 2294, 2295, 2296],
           dtype='int64', length=2268)

In [8]:
for grb_idx in range(args.grb_name_idx_start, args. grb_name_idx_end):
    grb_name = df.grb_name[grb_idx]
    probs = np.maximum(1e-15, np.load(DATA_DIR+"/grbwebgbm/healpix/{}_healpix_nside64.npy".format(grb_name)))
    
    with time("{}_tw{}".format(grb_name, args.tw_in_second)):
        TSs = []
        for i, file in enumerate(files):
            if i % (len(files)/10) == 0:
                print("Working on number of file: {}".format(i))
            scans = sparse.load_npz(file)
            for idx in range(scans.shape[0]):
                scan = scans[idx]
                TS = getCombinedMaxTS(probs, scan)
                TSs.append(TS)
                
    
    sTSs = sparse.csr_matrix(TSs, dtype=float)
    
    outfilename = "{}_tw{}_NTrial{}.npz".format(grb_name, 
                                                 args.tw_in_second,
                                                sTSs.shape[1])
    output_folder = cy.utils.ensure_dir(ANA_DIR+"/allsky_scan/with_prior_background/tw{}".format(args.tw_in_second))
#     sparse.save_npz(output_folder+"/{}".format(outfilename) ,sTSs)
 

Working on number of file: 0

0:00:04.789334 elapsed.
Working on number of file: 0

0:00:04.358086 elapsed.
